In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import osmnx as ox

In [2]:
#read in road network
road = gpd.read_file('../../../data/oradell/osm/osm-roads/streets-with-osm-way-type.shp')
road

,street_edg,osm_id,way_type,geometry
0,87,11562297,residential,"LINESTRING (-74.03996 40.94794, -74.04089 40.9..."
1,184,11562297,residential,"LINESTRING (-74.03088 40.95201, -74.03136 40.9..."
2,477,11564127,residential,"LINESTRING (-74.03435 40.95815, -74.03432 40.9..."
3,273,11564531,residential,"LINESTRING (-74.03670 40.94918, -74.03755 40.9..."
4,51,11564531,residential,"LINESTRING (-74.02456 40.94985, -74.02463 40.9..."
...,...,...,...,...
484,465,1027007129,residential,"LINESTRING (-74.02582 40.94882, -74.02580 40.9..."
485,449,1027007129,primary,"LINESTRING (-74.03218 40.95121, -74.03216 40.9..."
486,64,1027007129,residential,"LINESTRING (-74.02885 40.94212, -74.02929 40.9..."
487,55,1027007129,residential,"LINESTRING (-74.03680 40.95124, -74.03641 40.9..."


In [4]:
#read in labels
labels = gpd.read_file('../../../data/oradell/processed-labels/labels_raw_oradell/labels_raw_oradell.shp')
labels

,audit_task,label_id,gsv_panora,label_type,severity,correct,high_quali,gsv_pano_1,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry
0,7,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111,POINT (-74.03411 40.94552)
1,7,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142,POINT (-74.03414 40.94554)
2,7,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157,POINT (-74.03416 40.94543)
3,7,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203,POINT (-74.03420 40.94545)
4,7,13,8AsY4nlOJQAo8P5YYaEjzA,Crosswalk,1.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,284.392853,-13.758928,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945473,-74.034142,POINT (-74.03414 40.94547)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,2194,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,1,1,28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624,POINT (-74.03062 40.95438)
12130,2194,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,None,1,28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617,POINT (-74.03062 40.95439)
12131,2194,13098,5zDLTjYdw6hQhfsXppE0cA,SurfaceProblem,1.0,1,1,5zDLTjYdw6hQhfsXppE0cA,2,77.437500,-23.928572,287.278870,-2.803400,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954624,-74.030891,POINT (-74.03089 40.95462)
12132,2202,13099,WkdNrR64GZUC1V_at1pxyA,NoCurbRamp,NaN,None,0,WkdNrR64GZUC1V_at1pxyA,1,178.687500,-8.625000,281.559601,-2.859410,0cfe6980-35ac-4cdd-a2df-cbee02531406,40.969189,-74.029976,POINT (-74.02998 40.96919)


In [5]:
#change coordinates
labels = labels.to_crs(epsg=3424)
road = road.to_crs(epsg=3424)

In [6]:
#spatial join project sidewalk labels to road geometry
labels_mapped_on_road = labels.sjoin_nearest(road, distance_col='distance_to_road')
labels_mapped_on_road

,audit_task,label_id,gsv_panora,label_type,severity,correct,high_quali,gsv_pano_1,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,index_right,street_edg,osm_id,way_type,distance_to_road
0,7,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111,POINT (620819.055 769691.165),34,43,11566869,residential,8.719586
1,7,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142,POINT (620810.580 769699.458),34,43,11566869,residential,13.715870
3,7,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203,POINT (620793.905 769664.625),34,43,11566869,residential,24.682769
4,7,13,8AsY4nlOJQAo8P5YYaEjzA,Crosswalk,1.0,1,1,8AsY4nlOJQAo8P5YYaEjzA,2,284.392853,-13.758928,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945473,-74.034142,POINT (620810.714 769674.443),34,43,11566869,residential,9.825799
23,7,34,7gUjKaABQ1OO0_nHdEIiUg,NoSidewalk,5.0,1,1,7gUjKaABQ1OO0_nHdEIiUg,1,281.687500,-3.660714,289.336060,-2.632355,503d2482-420f-454e-8118-aac2d22592aa,40.945621,-74.034348,POINT (620753.522 769728.340),34,43,11566869,residential,21.937413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,1709,12493,dkjDwJszcttKtjEswrxR_w,CurbRamp,NaN,1,1,dkjDwJszcttKtjEswrxR_w,1,96.000000,-2.000000,203.994247,0.323746,89f4e8c9-816d-4a4e-b6d3-4f5c17bbf3e7,40.941204,-74.028244,POINT (622448.206 768128.070),74,356,11571687,residential,8.838842
11855,1709,12494,dkjDwJszcttKtjEswrxR_w,SurfaceProblem,3.0,None,1,dkjDwJszcttKtjEswrxR_w,1,96.000000,-2.000000,203.994247,0.323746,89f4e8c9-816d-4a4e-b6d3-4f5c17bbf3e7,40.941204,-74.028244,POINT (622448.206 768128.070),74,356,11571687,residential,8.838842
11908,1753,12570,xSWQF-ZBbSvFSHU0uyMcLQ,Crosswalk,5.0,0,1,xSWQF-ZBbSvFSHU0uyMcLQ,3,89.263390,-11.859375,289.119690,-9.943140,85eb67be-6d90-491d-9150-41b2a1775a71,40.943504,-74.029350,POINT (622138.087 768964.432),275,493,11585828,primary,22.739192
11909,1753,12571,xSWQF-ZBbSvFSHU0uyMcLQ,NoCurbRamp,3.0,0,1,xSWQF-ZBbSvFSHU0uyMcLQ,3,89.263390,-11.859375,289.119690,-9.943140,85eb67be-6d90-491d-9150-41b2a1775a71,40.943497,-74.029320,POINT (622146.532 768961.698),275,493,11585828,primary,13.868833


In [8]:
#select only label_id, distance
distance = labels_mapped_on_road[['label_id','way_type', 'distance_to_road']]

In [9]:
#drop duplicates
distance = distance.drop_duplicates(subset=['label_id'], keep='first')

In [10]:
# reset index
distance = distance.reset_index(drop=True)

In [12]:
distance

,label_id,way_type,distance_to_road
0,9,residential,8.719586
1,10,residential,13.715870
2,12,residential,24.682769
3,13,residential,9.825799
4,34,residential,21.937413
...,...,...,...
12129,12493,residential,8.838842
12130,12494,residential,8.838842
12131,12570,primary,22.739192
12132,12571,primary,13.868833


In [16]:
# save distance into a csv file
distance.to_csv('../../../data/oradell/features/distance_to_road.csv', index=False)